### Import packages

In [1]:
import os
import googleapiclient.discovery
import urllib.parse as p

In [ ]:
from dotenv import load_dotenv # add this line
load_dotenv()

### Set-up API

In [2]:
# Disable OAuthlib's HTTPS verification when running locally.
# *DO NOT* leave this option enabled in production.

api_service_name = "youtube"
api_version = "v3"
DEVELOPER_KEY = os.getenv('DEVELOPER_KEY')

os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"
youtube = googleapiclient.discovery.build(
    api_service_name, api_version, developerKey = DEVELOPER_KEY)

### Define functions for API calls

In [3]:
def get_uploads_id(youtube, channel_id):
    request = youtube.channels().list(
            part="snippet,contentDetails",
            id=channel_id
        )
    response = request.execute()
    
    return response['items'][0]['contentDetails']['relatedPlaylists']['uploads']

def get_uploaded_videos_response(youtube, playlist_id):
    params = {
        "part":"contentDetails",
        "playlistId": playlist_id,
        "maxResults": 50
    }

    uploaded_videos_content_details_list = []

    while True:
        response = youtube.playlistItems().list(**params).execute()
        uploaded_videos_content_details_list = response.get('items') + uploaded_videos_content_details_list

        if 'nextPageToken' in response.keys():
            params['pageToken'] = response['nextPageToken']
            continue
        else:
            break

    return uploaded_videos_content_details_list

def get_comments_from_videos(youtube, video_content_details):
    
    comments_from_video = {}
    
    for video in video_content_details:
        video_id = video['contentDetails']['videoId']
        params = {
            'videoId': video_id, 
            'maxResults': 5,
            'order': 'relevance', # default is 'time' (newest)
        }
        comments_from_video[video_id] = []

        while True:
            response = youtube.commentThreads().list(part="snippet", **params).execute()
            comments_from_video[video_id] = response.get('items') + comments_from_video[video_id]
            if 'nextPageToken' in response.keys():
                params['pageToken'] = response['nextPageToken']
                continue
            else:
                break

        return comments_from_video

### Call data APIs

In [5]:
channel_id = "UCamLstJyCa-t5gfZegxsFMw"

In [6]:
playlist_id = get_uploads_id(youtube, channel_id)

In [7]:
uploaded_videos_response = get_uploaded_videos_response(youtube, playlist_id)

In [8]:
comments_from_all_uploads = get_comments_from_videos(youtube, uploaded_videos_response)

## Save results

In [10]:
import json

In [ ]:
with open('uploaded_videos_response.json', 'w', encoding='utf-8') as f:
    data = {
                "uploaded_videos_response": uploaded_videos_response
            }
    json.dump(data, f, ensure_ascii=False, indent=4)

In [ ]:
with open('comments_from_all_uploads.json', 'w', encoding='utf-8') as f:
    json.dump(comments_from_all_uploads, f, ensure_ascii=False, indent=4)

In [ ]:
#channel_id = "UCamLstJyCa-t5gfZegxsFMw"
#playlist_id = "UUamLstJyCa-t5gfZegxsFMw"
f1 = open('uploaded_videos_response.json')
f2 = open('comments_from_all_uploads.json')

j1 = json.load(f1)
j2 = json.load(f2)